# Nigeria Rivers

Showing Guernsey's beaches on a map

In [1]:
import requests
import json
from types import SimpleNamespace

import pickle
import os
import re

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool
from bokeh.models import LabelSet
from bokeh.palettes import viridis, Category20, Blues256

output_notebook()

from pyproj import Transformer

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [4]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [5]:

file_name = 'data/kvs'

if os.path.isfile(file_name):
    # open a file, where you stored the pickled data
    file = open(file_name, 'rb')

    # dump information to that file
    kvs = pickle.load(file)

    # close the file
    file.close()

else:
    kvs = osm.toolkit.get_osm_kvs()  
        
    # open a file, where you ant to store the data
    file = open(file_name, 'wb')

    # dump information to that file
    pickle.dump(kvs, file)

    # close the file
    file.close()


In [6]:
tags = Tags(kvs)

In [7]:
area_Guernsey = "(49.4096, -2.6875,49.5194, -2.4901)"
area_Ealing = "(51.487, -0.351, 51.5335, -0.2655)"
area_britishisles = "(48.327, -12.063, 61.428, 1.78)"
area_Glasgow = "(55.5683, -4.9507, 56.1318, -3.6639)"
area_Whitelee = "(55.5, -4.4364, 55.7591, -4.1)"
area_Wales = "(51.221, -5.559, 53.619, -2.845)"
area_Shetlands = "(59.8034, -2.4527, 60.9331, -0.4257)"
area_London = "(51.448, -0.2554, 51.6061, 0.0714)"
area_Nigeria = "(2,2,15,15)"
# area_Nigeria = "(9,4.6,10,4.8)" # used for testing
area_Nigeria = "(-40,-20,20,55)"

In [8]:
df_hydro = osm.toolkit.get_osm_data(key=tags.plant__source,tag=tags.plant__source.hydro,area=area_Nigeria,output="center")


In [9]:
df_hydro["x"], df_hydro["y"] = TRANSFORM_4326_TO_3857.transform(df_hydro["lat"], df_hydro["lon"])
df_hydro["coordinates"] = tuple(zip(df_hydro["lat"], df_hydro["lon"]))

df_hydro = df_hydro.rename(columns=lambda x: re.sub(':','_',x))

df_hydro_short = df_hydro[["x","y","name","plant_output_electricity","plant_method","coordinates","id"]]

source_hydro = ColumnDataSource(df_hydro_short)

In [10]:
# df_nigeria = osm.toolkit.get_osm_data_by_id(member_id=192787,obj_type="rel",output="geom")

# df_nigeria_border = pd.DataFrame.from_records(df_nigeria['members'][0])


In [11]:
# df_nigeria_border = df_nigeria_border[df_nigeria_border["type"]=="way"]
# df_nigeria_border = df_nigeria_border[df_nigeria_border["role"]=="outer"]

In [12]:
# # create the border
# xs = []
# ys = []

# for row in df_nigeria_border.iterrows():
#     way=pd.DataFrame.from_records(row[1]['geometry'])
       
#     x,y = TRANSFORM_4326_TO_3857.transform(way['lat'],way['lon'])   
    
#     xs.append(x)
#     ys.append(y)
        
# source_border = ColumnDataSource({'xs':xs, 'ys':ys}) 

In [13]:
# df_rivers = osm.toolkit.get_osm_data(key=tags.waterway,tag=tags.waterway.river,area=area_Nigeria,output="geom",element="rel")
# 

In [14]:
# df_rivers = df_rivers[df_rivers["type"]=="waterway"]
# df_rivers = df_rivers.dropna(subset=['name'])

# names = set(df_rivers['name'].dropna())

# color_palette = list()
# for i in range(len(names)):
#     color_palette.append(Blues256[i%255])

# color_mapping = dict(zip(names,color_palette))

# df_rivers["color"] = df_rivers['name'].map(color_mapping)



In [15]:
# # create the rivers
# xs = []
# ys = []
# names = []
# colors = []

# for cnt,row in df_rivers.iterrows():
#     #print("====row")
#     #print(row)

#     df_river_ways = pd.DataFrame.from_records(row['members'])

#     for cnt2,way in df_river_ways.iterrows():
#         #print("====way")
#         #print(way)
#         if way["type"]=="way":
            
#             if isinstance(way['geometry'], list):
#                 geom=pd.DataFrame.from_records(way['geometry'])
#             else:
#                 geom=way['geometry']

#             x,y = TRANSFORM_4326_TO_3857.transform(geom['lat'],geom['lon'])   
        
#             xs.append(x)
#             ys.append(y)
#             names.append(row["name"])
#             colors.append(row["color"])
        
# source_rivers = ColumnDataSource({'xs':xs, 'ys':ys, "name":names, "color":colors}) 

In [16]:
output_file("Africa_Hydro.html")

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

# Create a bokeh figure with tiles
plot_map = figure(width=800, 
                  height=800,
                  x_axis_type="mercator", 
                  y_axis_type="mercator",
                )

plot_map.add_tile(MAP_TILES['ESRI'])

# tooltips = [("name", "@name")]

# border = plot_map.multi_line(xs='xs',
#                            ys='ys',
#                            source=source_border,
#                            line_width=6,
#                            line_color='yellow')
    



TileRenderer(id='p1035', ...)

In [17]:
# render_rivers = plot_map.multi_line(xs='xs',
#                  ys='ys',
#                  source=source_rivers,
#                  line_width=2,
#                  line_color='color',
#           )

# hover_rivers = HoverTool(renderers=[render_rivers])
# hover_rivers.tooltips = [("Name", "@name")]
# plot_map.add_tools(hover_rivers)

In [18]:
render_hydro = plot_map.scatter(x="x", y="y", source=source_hydro, size=8, color="blue")

hover_hydro = HoverTool(renderers=[render_hydro])
hover_hydro.tooltips = [("Name", "@name"),("Capacity", "@plant_output_electricity"),("Type", "@plant_method"),("Location", "@coordinates")]
plot_map.add_tools(hover_hydro)

render_hydro.selection_glyph = None
render_hydro.nonselection_glyph = None

url = "https://www.openstreetmap.org/way/@id/"
render_hydro.selection_glyph = None
render_hydro.nonselection_glyph = None
taptool_hydro = TapTool(renderers=[render_hydro])
taptool_hydro.callback = OpenURL(url=url)
plot_map.add_tools(taptool_hydro)

In [19]:
# area_Nigeria

In [20]:
from bokeh.models import Range1d

x,y = TRANSFORM_4326_TO_3857.transform([-40,20],[-20,55])

# set a range using a Range1d
plot_map.x_range = Range1d(x[0], x[1])
plot_map.y_range = Range1d(y[0], y[1])

plot_map.title.text = "The 301 hydro power plants in sub-saharan Africa extracted from OpenStreetMap"

In [21]:
show(plot_map)

In [22]:
len(df_hydro)

301